<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [5]:
## Task 1: Mark all launch sites on a map
## can't access the link, download the data instead
spacex_df = pd.read_csv("spacex_launch_geo.csv")

# normalize column names, just in case
spacex_df = spacex_df.rename(columns={
    'LaunchSite': 'Launch Site',
    'Latitude': 'Lat',
    'Longitude': 'Long',
    'Class': 'class'
})

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
launch_sites_df = (
    spacex_df.groupby('Launch Site', as_index=False)
             .first()[['Launch Site','Lat','Long']]
)

site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)

for _, row in launch_sites_df.iterrows():
    name, lat, lon = row['Launch Site'], float(row['Lat']), float(row['Long'])
    folium.Circle([lat, lon], radius=1000, color='#d35400', fill=True)\
          .add_child(folium.Popup(name)).add_to(site_map)
    folium.Marker([lat, lon],
                  icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                               html=f'<div style="font-size:12px;color:#d35400;"><b>{name}</b></div>')
                 ).add_to(site_map)

site_map

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
launch_sites_df = (
    spacex_df.groupby('Launch Site', as_index=False)
             .first()[['Launch Site', 'Lat', 'Long']]
)

# Initialize the map (centered at NASA JSC or any center you prefer)
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle + popup label for each site
for _, row in launch_sites_df.iterrows():
    name = row['Launch Site']
    lat, lon = float(row['Lat']), float(row['Long'])

    folium.Circle(
        location=[lat, lon],
        radius=1000,
        color='#d35400',
        weight=2,
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.15,
        popup=folium.Popup(name, max_width=250)
    ).add_to(site_map)

    folium.Marker(
        location=[lat, lon],
        icon=folium.features.DivIcon(
            icon_size=(140, 24),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px;color:#d35400;"><b>{name}</b></div>'
        )
    ).add_to(site_map)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


Proximity to the Equator: No, the SpaceX launch sites are not located directly on the Equator.
The closest site is Cape Canaveral (Florida) at a latitude of about 28.5°N.
The Vandenberg site in California is even farther north at around 34.6°N.

Although not equatorial, these sites are still relatively close to the Equator compared to other U.S. latitudes. The reason for choosing latitudes closer to the Equator is Earth’s rotational boost: rockets launched eastward near the Equator gain extra velocity due to Earth’s spin, which makes launches more fuel-efficient.

Proximity to the Coast: Yes, all SpaceX launch sites are located very close to the coastline.
Cape Canaveral and Kennedy Space Center are on the east coast of Florida, right by the Atlantic Ocean.
Vandenberg Air Force Base is on the California coast, facing the Pacific Ocean.

The main reason is safety and trajectory. Launching over the ocean minimizes the risk to populated areas in case of accidents and allows safe disposal of rocket stages or debris into open water. It also provides unobstructed flight paths for different orbital inclinations.

In summary, SpaceX launch sites are strategically located near the coastlines at relatively low latitudes to balance efficiency, safety, and U.S. geographical limitations.

In [11]:
# --- Task 2: Mark the success/failed launches for each site on the map ---

# 0) Safety check: make sure the expected columns exist
required = {'Launch Site', 'Lat', 'Long', 'class'}
missing = required - set(spacex_df.columns)
if missing:
    raise ValueError(f"Missing columns in spacex_df: {missing}. Rename your columns to match {required}.")

# 1) Reuse your existing 'site_map' if it exists; otherwise initialize a fresh one
if 'site_map' not in globals():
    center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
    site_map = folium.Map(location=center, zoom_start=5)

# 2) Create a MarkerCluster to group many markers at the same coordinate
from folium.plugins import MarkerCluster
cluster = MarkerCluster(name="Launch Outcomes").add_to(site_map)

# 3) Create a new column 'marker_color' based on the class value (1=success -> green, 0=failure -> red)
def _color_from_outcome(v):
    try:
        return 'green' if int(v) == 1 else 'red'
    except Exception:
        return 'gray'  # just-in-case fallback

spacex_df['marker_color'] = spacex_df['class'].apply(_color_from_outcome)

# 4) Add a marker for each launch record to the cluster
for _, r in spacex_df.iterrows():
    lat, lon = float(r['Lat']), float(r['Long'])
    site = r['Launch Site']
    outcome = int(r['class'])  # 1 or 0

    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=r['marker_color'], icon='ok-sign'),
        popup=folium.Popup(f"<b>Site:</b> {site}<br><b>Outcome:</b> {outcome}", max_width=300)
    ).add_to(cluster)

# 5) (Optional) Layer control if you add more layers later
folium.LayerControl(collapsed=False).add_to(site_map)

# 6) (Optional) A tiny fixed legend like in the screenshot
legend_html = """
<div style="
  position: fixed; bottom: 30px; left: 30px; z-index: 9999;
  background: white; padding: 8px 10px; border: 1px solid #bbb;
  font-size: 12px; border-radius: 4px;">
  <b>Legend</b><br>
  <span style="display:inline-block;width:12px;height:12px;background:#28a745;margin-right:6px;"></span>
  Success (class=1) <br>
  <span style="display:inline-block;width:12px;height:12px;background:#dc3545;margin-right:6px;"></span>
  Failure (class=0)
</div>
"""
site_map.get_root().html.add_child(folium.Element(legend_html))

site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [21]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [19]:
# marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [20]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column 'marker_color' based on 'class' values
# spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Quick check
# spacex_df[['Launch Site', 'Lat', 'Long', 'class', 'marker_color']].head()

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [1]:
# Add marker_cluster to current site_map
# Create the cluster container

# Add markers for each launch


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
mousepos = MousePosition(
    position='topleft',
    prefix='Lat/Lon:',
    separator=' , ',
    num_digits=6,
    empty_string='Move mouse on map'
)
site_map.add_child(mousepos)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

site_map.add_child(folium.LatLngPopup())

import re

# Unique launch sites (one record per site)
launch_sites_df = (
    spacex_df.groupby('Launch Site', as_index=False)
    .first()[['Launch Site', 'Lat', 'Long']]
)

# Paste the coordinates you got from clicking the map popup
coords_str = "28.56367,-80.57163"   # <-- replace with your clicked coast point
lat_str, lon_str = re.split(r"[,\s]+", coords_str.strip())[:2]
coastline_lat, coastline_lon = float(lat_str), float(lon_str)

# Find the nearest launch site automatically
dists_km = launch_sites_df.apply(
    lambda r: calculate_distance(r['Lat'], r['Long'], coastline_lat, coastline_lon), axis=1
)
nearest_idx = dists_km.idxmin()

launch_site_name = launch_sites_df.loc[nearest_idx, 'Launch Site']
launch_site_lat  = float(launch_sites_df.loc[nearest_idx, 'Lat'])
launch_site_lon  = float(launch_sites_df.loc[nearest_idx, 'Long'])

# Distance in km
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Nearest launch site: {launch_site_name} @ ({launch_site_lat:.5f}, {launch_site_lon:.5f})")
print(f"Coastline point:     ({coastline_lat:.5f}, {coastline_lon:.5f})")
print(f"Distance to coast:   {distance_coastline:.2f} km")

Nearest launch site: CCAFS SLC-40 @ (28.56320, -80.57682)
Coastline point:     (28.56367, -80.57163)
Distance to coast:   0.51 km


In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Optional: group so you can toggle later via LayerControl
fg_coast = folium.FeatureGroup(name='Coastline distance', show=True)
site_map.add_child(fg_coast)

# Tiny highlight on the launch site (helps when zoomed in)
folium.Circle(
    location=[launch_site_lat, launch_site_lon],
    radius=180, color='#d35400', fill=True, fill_opacity=0.15
).add_to(fg_coast)

# Distance label at the coastline point (like the screenshot)
folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=folium.DivIcon(
        icon_size=(80, 18),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
).add_to(fg_coast)

site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# Optional: put each measurement in its own FeatureGroup for toggling

line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2
)

# Add to the same FeatureGroup so it toggles together with the label/markers
fg_coast.add_child(line)

# Optional: make sure the map view fits the two points nicely
site_map.fit_bounds([[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]])

# Optional: show a layer control if you haven't already
folium.LayerControl(collapsed=False).add_to(site_map)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [18]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

import re

# Helper: parse "lat,lon" or "lat lon"
def _to_latlon(s):
    a, b = re.split(r"[,\s]+", s.strip())[:2]
    return float(a), float(b)

# Helper: add a labeled distance marker + polyline from the launch site to any target point
def add_measurement(feature_name, target_lat, target_lon, color='#1E90FF'):
    fg = folium.FeatureGroup(name=feature_name, show=True).add_to(site_map)
    dist_km = calculate_distance(launch_site_lat, launch_site_lon, target_lat, target_lon)

    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [target_lat, target_lon]],
        weight=2, color=color, tooltip=f"{feature_name}: {dist_km:.2f} KM"
    ).add_to(fg)

    folium.Marker(
        location=[target_lat, target_lon],
        icon=folium.DivIcon(
            icon_size=(90, 18), icon_anchor=(0, 0),
            html=f'<div style="font-size:12px; color:{color};"><b>{dist_km:.2f} KM</b></div>'
        )
    ).add_to(fg)
    return dist_km

# ==== PASTE THE THREE COORDINATES YOU READ WITH MousePosition/LatLngPopup ====
# (Examples shown match the screenshots: NASA Railroad, Samuel C Phillips Pkwy, Melbourne, FL)
railway_coords = "28.56305,-80.58490"      # e.g., click on "NASA Railroad" label/line
highway_coords = "28.56540,-80.58420"      # e.g., click on "Samuel C Phillips Pkwy"
city_coords    = "28.08363,-80.60811"      # e.g., click near the "Melbourne" city label
# ============================================================================

rail_lat, rail_lon = _to_latlon(railway_coords)
hwy_lat,  hwy_lon  = _to_latlon(highway_coords)
city_lat, city_lon = _to_latlon(city_coords)

# Draw/label distances (colors chosen to be distinct)
d_rail = add_measurement("Railway distance", rail_lat, rail_lon, color="#10B981")
d_hwy  = add_measurement("Highway distance", hwy_lat, hwy_lon, color="#F59E0B")
d_city = add_measurement("City distance",    city_lat, city_lon, color="#EF4444")

# Optional: fit the map to include the furthest of the three targets
site_map.fit_bounds([
    [launch_site_lat, launch_site_lon],
    [rail_lat, rail_lon], [hwy_lat, hwy_lon], [city_lat, city_lon]
])

folium.LayerControl(collapsed=False).add_to(site_map)
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


1) Are launch sites in close proximity to railways?

Yes. Around Cape Canaveral the NASA Railroad runs right through the range—your clicks should place it roughly hundreds of meters to ~1–2 km from LC-40/LC-39 pads. Rail provides heavy-cargo access (boosters, propellants, ground hardware) that’s cheaper and safer than moving everything by road.

2) Are launch sites in close proximity to highways?

Yes. Pads tie into purpose-built access roads and nearby highways (e.g., Samuel C. Phillips Pkwy at CCAFS). Distances are again sub-kilometer to a couple of kilometers. That short hop is intentional: fast access for crews, fire/rescue, fueling convoys, and routine pad turnover—without routing heavy trucks through dense neighborhoods.

3) Are launch sites in close proximity to the coastline?

Yes—very. Both Florida (CCAFS/KSC) and Vandenberg sit right on the coast. Your coastline line should come out on the order of ~0.5–2 km for Florida and similarly small at Vandenberg. Coastal siting allows over-ocean flight paths, so any debris, jettisoned hardware, or off-nominal events fall into designated marine zones rather than populated areas. It also gives Florida an eastward corridor for low-inclination/ISS launches and Vandenberg a southward corridor for polar orbits.

4) Do launch sites keep a certain distance away from cities?

Yes—tens of kilometers. Your city measurement (e.g., to Melbourne, FL) should land in the ~30–60 km range depending on the city center pin you chose. Pads are deliberately separated from population centers to reduce risk, sonic boom and acoustic exposure, and to create large exclusion zones for countdowns and launches.

Why these patterns make sense:
* Safety first: Coastal placement + downrange ocean corridors minimize risk to people and property; standoff from cities further reduces exposure to blast overpressure, debris, and toxic plumes during worst-case scenarios.
* Logistics: Rail and highway proximity enables reliable delivery of large, heavy, or hazardous cargo and quick access for emergency response—all while keeping heavy traffic out of city cores.
* Performance & trajectories: Florida’s lower latitude (≈28.5° N) gives a rotation boost for eastbound launches, improving payload to orbit. Vandenberg’s coast and geography provide clear polar/sun-synchronous routes over the Pacific with minimal overflight.
* Regulatory practicality: Coastal, sparsely populated ranges simplify airspace and maritime closures, environmental constraints, and public safety planning.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
